# Contact Flux

In this chapter we will calculate the contactflux at the contact point.

The script is here {download}`Script <./F.pdf>`.

In [1]:
# Load packages
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox, VBox
#%matplotlib ipympl

In [2]:
def conT(T, b2):
    T_1 = 35
    T_2 = T
    b1  = (0.5 * 1000 * 4000)**0.5            # human skin
    T_c      = (b1 * T_1 + b2 * T_2) / (b1 + b2)
    return T_c
@widgets.interact(b2=(0, 2000))#_manual
def plot(b2, grid=True):
    T = np.linspace(0., 50, 100)
    fig, ax = plt.subplots(1, 1, figsize=(7, 3))
    ax.plot(T, conT(T, b2))
    ax.set_xlabel('T second media')
    ax.set_ylabel('Contact Temperature')
    ax.grid(grid)

interactive(children=(IntSlider(value=1000, description='b2', max=2000), Checkbox(value=True, description='gri…

```{admonition} Questions
:class: full-width
 
Do you understand the influence of the individual variables? 

If not, use the rocket and calculate with other values.;-)!
```

Now follows the numerical calculation with the forward step method of Euler (explicit forward). For more about it look at [Euler](https://aquaulb.github.io/book_solving_pde_mooc/solving_pde_mooc/notebooks/02_TimeIntegration/02_01_EulerMethod.html).

```{admonition} Questions
:class: full-width
 
Before we discuss the programme code, try to understand what is shown in the diagram:

- what happens at the contact point?
- What do you observe in terms of temperature changes at the edges?
- are both ok, or does physics speak against it?
```

In [3]:
# Discretitations
Nx = 50 # first medium
Ny = 50 # second medium
Nt = 600
dt = 0.1

# Initial temperatures
T1 = np.zeros((Nx, Nt)) + 36
T2 = np.zeros((Ny, Nt)) + 20

# Thermal diffusivities
lam_1 = 1
rhocp_1 = 1000 * 4000
lam_2 = 0.04
rhocp_2 = 100 * 1000
a_1 = lam_1 / rhocp_1
a_2 = lam_2 / rhocp_2

d1 = 0.1
dx = d1 / (Nx - 1)
d2 = 0.1
dy = d2 / (Ny - 1)
x  = np.linspace(0, d1, Nx)
y  = np.linspace(d1, d1+d2, Ny)
f1 = np.zeros(Nt)
f2 = np.zeros(Nt)


def dTdt(t, x, alpha, a_lam, e_lam, e_lam2, n_lam, lam_B, r_cv_d, r_cv_n, delta, Ke):         
    K = 5
    
    T_B = x[0: K]
    T_L = x[K: K + Ke]
    T_BL = x[K + Ke: K + Ke + 1]
    
    dT_L = np.zeros(Ke)
    dT_B = np.zeros(K)
    dT_BL = np.zeros(1)
    
    dB = 0.2     # Dicke der Bodenschicht
    dL = 1000    # Höhe der berücksichtigten Luftschicht
    c_B = 1500 * 1000
    c_BL = 1500 * 1000
    dx_BL = 0.03
    dx_B = dB / K
    #lam_B = 0.55
    lam_Bt = lam_B #- 0.0 * np.cos(2*np.pi/24 * (t/3600))
    r_B = dx_B / lam_Bt
    r_r = 0.2 * 2
    c_L = 1.2 * 1000
    dx_L = dL / Ke
    
    if f_I(t) <= 10:
        I_s = 0
        lam_e = n_lam
        r_cv_v = r_cv_n #/ (1 + np.log(1 + f_v(t)))
        #r_B = 0.08
    else:
        I_s = f_I(t)
        r_cv_v = r_cv_d / (1 + np.log(1 + f_v(t)))
        #lam_e = a_lam * np.exp(e_lam/100 * abs(T_B[0] - T_L[0]))
       # lam_e = a_lam * np.exp(e_lam/100 * ((T_B[0] - f_TL(t)))**1) #T_L[0]))
    #print(lam_e)
        lam_e = a_lam * np.exp(e_lam/100 * (1 + 2*np.log(1 + f_v(t))))#lambda der Eddies
    r_e = dx_L / lam_e

    dT_BL[0] = (0.5 * I_s * alpha - 1/r_cv_v * (T_BL[0] - T_L[0]) 
            - 1/r_r * (T_BL[0] - T_H(T_L[0], f_phi(t)))) / (c_BL * dx_BL / 2)
    
    dT_B[0] = (0.5 * I_s * alpha - 1/r_B * (T_B[0] - T_B[1]) - 1/r_cv_v * (T_B[0] - T_L[0]) 
               #- 1/r_r * (T_B[0] - T_H(f_TL(t), f_phi(t)))) / (c_B * dx_B / 2)
                - 1/r_r * (T_B[0] - T_H(T_L[0], f_phi(t)))) / (c_B * dx_B / 2)
    #print(T_H(f_TL(t), f_phi(t)))
               #- 1/r_r * (T_B[0] - (T_L[0] - delta))) / (c_B * dx_B / 2)
    for k in range(1, K-1):
        dT_B[k] = lam_Bt / c_B / dx_B**2 * (T_B[k - 1]  - 2 * T_B[k] + T_B[k + 1])
    dT_B[K-1] = 0
        
    dT_L[0] = (1/r_cv_v *(T_B[0] - T_L[0]) - 1/r_e * (T_L[0] - T_L[1])) / (c_L * dx_L / 2)
    for k in range(1, Ke-1):
        dT_L[k] = lam_e / c_L / dx_L**2 * (T_L[k - 1]  - 2 * T_L[k] + T_L[k + 1])
    dT_L[Ke-1] = 0#- 5 * 2 / c_L / dx_L * 2 
    
    y = np.concatenate((dT_B, dT_L, dT_BL))
    #y = dT_L
    return y


# Euler forward

def dTdt():
    for nt in range(0, Nt-1):
        for nx in range(1, Nx-1):
            dT1[nx] = 1 / dx**2 * a_1 * (T1[nx+1, nt] + T1[nx-1, nt] - 2*T1[nx, nt])
        dT1[Nx-1] = 1 / ((dx * rhocp_1 + dy * rhocp_2) * 0.5) 
                    * (lam_1 / dx * (T1[Nx-2, nt] - T1[Nx-1, nt]) -  lam_2 / dy * (T2[0, nt] - T2[1, nt])))
        #T2[0, nt+1]  = T1[Nx-1, nt+1] # Contacttemperature
    #T2[0, nt+1] = T2[0, nt] + (dt / ((dx * rhocp_1 + dy * rhocp_2) * 0.5) 
     #               * (lam_1 / dx * (T1[Nx-2, nt] - T1[Nx-1, nt]) -  lam_2 / dy * (T2[0, nt] - T2[1, nt])))
    #for ny in range(1, Ny-1):
    #    T2[ny, nt+1] = T2[ny, nt] + dt / dy**2 * a_2 * (T2[ny+1, nt] + T2[ny-1, nt] - 2*T2[ny, nt])
    return dT1
#f1[:] = lam_1 / dx * (T1[-2, :] - T1[-1, :])
#f2[:] = lam_2 / dy * (T2[0, :] - T2[1, :])

# Design a figure with plot-axes (ax)
fig, ax = plt.subplots(figsize=(11, 5))
ax.plot(f1)
ax.plot(f2)
#ax.plot(x, T1[:, ::10], 'r-')
#ax.plot(y, T2[:, ::10], 'b-')
ax.set_ylabel('T /°C')
ax.set_xlabel('x / m')
Med1 = "Medium 1"
Med2 = "Medium 2"
#ax.text(0.25, .51, Med1, fontsize=11, transform=ax.transAxes)
#ax.text(0.75, .51, Med2, fontsize=11, transform=ax.transAxes)
ax.grid()
plt.show()

IndentationError: unexpected indent (1284092528.py, line 96)

In [ ]:
print('Contact-temperature =', f'{T1[-1, -1]:.2f}', '°C')

```{admonition} Exercises
:class: full-width

Now it's your turn! Modify 

- the discretisation in place and time
- the material properties and

write down what happens!

What is the contact temperature between skin and wood?

And between skin an glas?

What are the differences to the exactly calculated values from equation {eq}`Tc`.
```